In [23]:
# Importing the required libraries
import os
import cv2
import glob
import zipfile
import numpy as np
from tqdm import tqdm
from urllib import request
#from google.colab import drive

In [24]:
#you need to replace this by the location of the dataset on your drive
# datasetloc="/content/drive/MyDrive/Hemangini Projects/dataset"
#! wget -q https://storage.googleapis.com/drive-bulk-export-anonymous/20220428T122708Z/4133399871716478688/aa76fc5c-9354-4b5f-ae56-aca2048e1129/1/acf3a9e4-29f2-415b-86b6-f7ba375e19ee?authuser -O dataset.zip
# drive.mount('/content/drive')
# datasetloc="/content/drive/MyDrive/Colab Notebooks/dataset"

# ! wget -q https://tinyurl.com/r5ut6tjx -O dataset.zip
# ! unzip -oqq dataset
# datasetloc="./dataset"


#if not os.path.exists("./dataset.zip"):
#    response = request.urlretrieve("https://drive.google.com/file/d/1z_B2zhcjl0heRbdRGUTQKwC82WqDjBLb/view?usp=sharing", "dataset.zip")
#    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
#        zip_ref.extractall(".")
#datasetloc="./dataset"

#-----------------------------------------------------------------------------------------------
# The dataset has been obtained from Kaggle with the dataset called Real and Fake Face Detection
# It has been published by the Computational Intelligence and Photography Lab
# Department of Computer Science, Yonsei University
# https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection


#drive.mount('/content/drive')
datasetloc = "D:/major_project/one/Identifying-Human-Edited-Images-master/data/real_and_fake_face"
#datasetloc = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/dataset"

In [25]:
# Return a list of path for the images in the folder as an array
fakefiles=glob.glob(datasetloc+"/training_fake/*.jpg")

In [26]:
#reading the FAKE images and labelling them as 1
#tqdm helps to keep track of progress
data=[]
y=[]
for f in tqdm(fakefiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  
  data.append(img)
  y.append(1)

100%|██████████| 960/960 [00:12<00:00, 77.69it/s] 


In [27]:
#realfiles=glob.glob(datasetloc+"/training_real/*.jpg")

In [28]:
realfiles=glob.glob(datasetloc+"/training_real1/*.jpg")

In [29]:
# Reading the REAL images and labelling them as 0
# Resize image so we can stay consistent with the uniformity throughout
for f in tqdm(realfiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  data.append(img)
  y.append(0)
  

100%|██████████| 960/960 [00:23<00:00, 41.14it/s]


In [30]:
#convering lists into arrays // Arrays of all pixelated arrays
data = np.asarray(data)
y = np.asarray(y)

In [31]:
# We do a random split on the dataset at 80% and 20% with; 80% of the data would be training the model and remaining 20% of the data would be checking the model performance
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [32]:
#importing the libraries required for our cnn model
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

In [33]:
# Sketched Model Architecture for FAKE an REAL image detection
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (128,128,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
# In the last layer if the value is Greater then 0.5 it is fake image and if the value is less then 0.5 it is real image

In [34]:
#fitting our model on the dataset, train the model using X train and Y train
model.fit(X_train,y_train,epochs=10,batch_size=40)

Epoch 1/10
39/39 [==============================] - 23s 571ms/step - loss: 21.5251 - accuracy: 0.4941
Epoch 2/10
39/39 [==============================] - 22s 568ms/step - loss: 0.6969 - accuracy: 0.5020
Epoch 3/10
39/39 [==============================] - 22s 569ms/step - loss: 0.6969 - accuracy: 0.5130
Epoch 4/10
39/39 [==============================] - 23s 583ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 5/10
39/39 [==============================] - 23s 589ms/step - loss: 0.6841 - accuracy: 0.5423
Epoch 6/10
39/39 [==============================] - 22s 565ms/step - loss: 0.6894 - accuracy: 0.5085
Epoch 7/10
39/39 [==============================] - 24s 610ms/step - loss: 0.6823 - accuracy: 0.5130
Epoch 8/10
39/39 [==============================] - 24s 607ms/step - loss: 0.6783 - accuracy: 0.5228
Epoch 9/10
39/39 [==============================] - 24s 604ms/step - loss: 0.6721 - accuracy: 0.5488
Epoch 10/10
39/39 [==============================] - 23s 595ms/step - loss: 0.6644 - accur

In [35]:
ypredtrain=model.predict(X_train)
ypredtest=model.predict(X_test)

12/12 [==============================] - 1s 100ms/step


In [36]:
# Using the train model we made predictions
yptrain=[]
yptest=[]
for y in ypredtrain:
  if(y>=0.5):
    yptrain.append(1)
  else:
    yptrain.append(0)
for y in ypredtest:
  if(y>=0.5):
    yptest.append(1)
  else:
    yptest.append(0)


In [37]:
from sklearn.metrics import accuracy_score
print("the accuracy score for training set is",accuracy_score(y_train, yptrain))
print("the accuracy score for test set is",accuracy_score(y_test, yptest))

the accuracy score for training set is 0.5755208333333334
the accuracy score for test set is 0.5182291666666666


In [ ]:
real = 'D:/major_project/one/Identifying-Human-Edited-Images-master/data/real_and_fake_face/training_real1/real_00015.jpg'
fake = 'D:/major_project/one/Identifying-Human-Edited-Images-master/data/real_and_fake_face/training_fake/easy_11_1111.jpg'